# Build a Neural network with Hyper-parameter fine tuning model

In [11]:
pip install tensorflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install keras==2.12.0

     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     -------------- ------------------------- 0.6/1.7 MB 19.5 MB/s eta 0:00:01
     ----------------------------------- ---- 1.5/1.7 MB 19.7 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 13.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.14.0 requires keras<2.15,>=2.14.0, but you have keras 2.12.0 which is incompatible.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
# Load the fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [6]:
# Build a simple neural network model
def create_model(hidden_units=128, learning_rate=0.01):
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(hidden_units, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [7]:
# Create a KerasClassifier
model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_8696\1833046629.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)


In [8]:
# Define the hyperparameters to search
param_grid = {
    'hidden_units': [64, 128, 256],
    'learning_rate': [0.001, 0.01, 0.1]
}

In [9]:
# Use GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_result = grid_search.fit(x_train, y_train)


500/500 [==============================] - 3s 6ms/step


In [10]:
# Evaluate the model with the best hyperparameters on the validation set
best_model = grid_result.best_estimator_
y_val_pred = best_model.predict(x_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy with Best Hyperparameters: {accuracy:.4f}")

375/375 [==============================] - 2s 5ms/step
Validation Accuracy with Best Hyperparameters: 0.8502


# Build an image classifier model with Pytorch 

In [2]:
pip install torch torchvision

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/74/07/edce54779f5c3fe8ab8390eafad3d7c8190fce68f922a254ea77f4a94a99/torch-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/20/ac/ab6f42af83349e679b03c9bb18354740c6b58b17dba329fb408730230584/torchvision-0.16.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ---------------------------------------- 0.2/192.3 MB 7.3 MB/s eta 0:00:27
   ---------------------------------------- 1.1/192.3 MB 13.9 MB/s eta 0:00:14
   ---------------------------------------- 1.7/192.3 MB 13.6 MB/s eta 0:00:15
   ---------------------------------------- 2.1/192.3 MB 13.3 MB/s eta 0:00:15
   ---------------------------------------- 2.3/192.3 MB 11.3 MB/s eta 0:00:17
    --------------------------------------- 2.4/192.3 MB 9.1 MB/s eta 0:00:21
    --------------------------------

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# Define transformations for the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image pixel values
])


In [4]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:50<00:00, 3409817.18it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# Create data loaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [6]:
# Define a simple convolutional neural network (CNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)  # Output size corresponds to the number of classes in CIFAR-10

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x


In [7]:
# Create an instance of the model
model = SimpleCNN().to(device)


In [8]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print training loss
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}')


Epoch 1/10, Loss: 1.2244
Epoch 2/10, Loss: 0.7349
Epoch 3/10, Loss: 0.6883
Epoch 4/10, Loss: 0.2125
Epoch 5/10, Loss: 0.3841
Epoch 6/10, Loss: 0.2447
Epoch 7/10, Loss: 0.3504
Epoch 8/10, Loss: 0.2914
Epoch 9/10, Loss: 0.2896
Epoch 10/10, Loss: 0.1228


In [10]:
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Test Accuracy: 71.97%
